Task 2 - Recommender System

In [12]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

Load the input file

In [13]:
file_folder = "Files"
file_name = "recom.csv"
path = f"{file_folder}//{file_name}"
df_recommender = pd.read_csv(path, header=0)
df_recommender.head()

,Unnamed: 0,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,ItemKey
0,0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0,5002.0
1,1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0,NaN
2,2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0,5005.0
3,3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0,NaN
4,4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,NaN


It looks like the Code_Product always equals the ItemKey, if the ItemKey as a value.
Need to check this in order to determine whether to remove the ItemKey column

In [14]:
def compare_Code_Product_with_ItemKey(df):
    print("Comparing the Code_Product column with the ItemKey column where the values are not null")
    df = df.dropna(subset="ItemKey")
    print(f"Comparing {len(df)} rows")
    result = df["Code_Product"].equals(df["ItemKey"])
    print(f"Both columns equal: {result}")

In [15]:
# df_recommender
compare_Code_Product_with_ItemKey(df_recommender)

Comparing the Code_Product column with the ItemKey column where the values are not null
Comparing 28597 rows
Both columns equal: True


Rename column names and convert data types. The Date column does not contain any useful information and can be removed. Drop the "ItemKey" column. Either it is the same as the "Code_Product" column or it is null 

In [16]:
rename_columns = {"Unnamed: 0" : "ID", "Main_ID": "Customer_ID", "Amount": "Order_Amount"}
df_recommender.rename(columns=rename_columns, inplace=True)
datatype_columns = {"Customer_ID" : "string", "Transaction_ID" : "string", "Order_Amount": int}
df_recommender = df_recommender.astype(datatype_columns)
#df_recommender.drop(["Date", "ItemKey"], axis=1, inplace=True)
df_recommender.info()
df_recommender.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              50000 non-null  int64  
 1   Customer_ID     50000 non-null  string 
 2   Transaction_ID  50000 non-null  string 
 3   Date            50000 non-null  object 
 4   Price           50000 non-null  float64
 5   Code_Product    50000 non-null  float64
 6   Order_Amount    50000 non-null  int32  
 7   ItemKey         28597 non-null  float64
dtypes: float64(3), int32(1), int64(1), object(1), string(2)
memory usage: 2.9+ MB


,ID,Customer_ID,Transaction_ID,Date,Price,Code_Product,Order_Amount,ItemKey
0,0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1,5002.0
1,1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1,NaN
2,2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1,5005.0
3,3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1,NaN
4,4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5,NaN


Pivot the dataframe for each customer

In [17]:
drop_columns = ["ID", "Date","Price", "ItemKey", "Transaction_ID"]
df = df_recommender.drop(columns=drop_columns)
df.head()


,Customer_ID,Code_Product,Order_Amount
0,90fada91,5002.0,1
1,9006f9ac,35012.0,1
2,32270891,5005.0,1
3,97e03e47,35078.5,1
4,41949228,49291.5,5


In [18]:
#df.groupby(["Customer_ID","Code_Product"], as_index=False)["Order_Amount"].sum().reset_index()
df = df.groupby(["Customer_ID","Code_Product"])["Order_Amount"].sum().reset_index()
df.head()

,Customer_ID,Code_Product,Order_Amount
0,00024de6,35097.0,2
1,00084856,49291.5,1
2,00084856,49292.0,1
3,0008e848,40002.0,1
4,00096930,49291.5,2


In [19]:
df.groupby(["Code_Product"])["Order_Amount"].sum().reset_index().sort_values("Order_Amount", ascending=False)


,Code_Product,Order_Amount
count,45901.000000,45901.000000
mean,32243.229026,1.342716
std,21345.751041,0.956183
min,5000.500000,1.000000
25%,10013.000000,1.000000
50%,40009.500000,1.000000
75%,49129.500000,1.000000
max,350027.500000,54.000000


In [20]:
df.describe()

,Code_Product,Order_Amount
count,45901.000000,45901.000000
mean,32243.229026,1.342716
std,21345.751041,0.956183
min,5000.500000,1.000000
25%,10013.000000,1.000000
50%,40009.500000,1.000000
75%,49129.500000,1.000000
max,350027.500000,54.000000


In [68]:
df_pivot = pd.pivot_table(df, index="Customer_ID", columns="Code_Product", values="Order_Amount")
df_pivot.head()


Code_Product,5000.5,5001.0,5001.5,5002.0,5002.5,5003.0,5004.0,5004.5,5005.0,5007.0,...,200035.5,200037.0,200037.5,200038.0,200038.5,200045.5,200046.0,200046.5,200047.0,350027.5
Customer_ID,,,,,,,,,,,,,,,,,,,,,
00024de6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00084856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0008e848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00096930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000c66b7,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
file_name = "recom_pivot.csv"
output_path = f"{file_folder}//{file_name}"
if os.path.exists(output_path):
    os.remove(output_path)
    
filepath = Path(output_path)
df_pivot.to_csv(filepath)